## Machine Learning
### Subgradient Method for Lasso Regression and Elastic Net
In **subgradient method**, we move in the negative of subgradient of the loss function in order to find the parameters. So, if the loss function is $L(\boldsymbol{w})$, then we update the parameter vector $\boldsymbol{w}$ by the subgradient of $L(\boldsymbol{w})$, denoted by $\partial L(\boldsymbol{w})$:
<br>$\boldsymbol{w}\leftarrow \boldsymbol{w}-\eta_k\partial L(\boldsymbol{w})$
<br> where $\eta_k>0$ is the **learning rate** (also called *step size*) at time step $k$.
<br>In **Elastic Net**, we use the following loss function:
<br>$L_{EN}(\boldsymbol{w})=\frac{1}{2}||\boldsymbol{y}-X\boldsymbol{w}||^2+\lambda_1 ||\boldsymbol{w}||_1+\frac{\lambda_2}{2} ||\boldsymbol{w}||^2$
<br>**Hint:** If we set $\lambda_2$ to zero, we get to the **Lasso**:
<br>$L_{Lasso}(\boldsymbol{w})=\frac{1}{2}||\boldsymbol{y}-X\boldsymbol{w}||^2+\lambda ||\boldsymbol{w}||_1$
<br>Now, we compute $\partial L_{EN}(\boldsymbol{w})$ by:
<br>$\partial L_{EN}(\boldsymbol{w})=-X^T(\boldsymbol{y}-X\boldsymbol{w})+\lambda_1 \partial ||\boldsymbol{w}||_1+\lambda_2 \boldsymbol{w}$
<br> such that
<br> $\partial ||\boldsymbol{w}||_1=[\partial |w_0|,\partial |w_1|,...,\partial |w_{q-1}| ]^T$
<br> where $\partial |w_i|=sign(w_i)$ if $w_i\ne0$; otherwise $[-1,1]$ 
<br><br>**Reminder:** We have data points $(\boldsymbol{x}_i,y_i)$ where the first components of $\boldsymbol{x}_i$ are one. Thus, the rows of matrix $X$ are composed of $\boldsymbol{x}^T_i$ such that the first column of $X$ is all one. Vectors are denoted here by bold symbols, and they are all column vectors.
<br><br>In the following, we download the file *diabetes.csv*, which is our dataset, composing of 768 rows and 9 columns. Its last column holds the values of $y_i$, while the rest of columns holds the values of $\boldsymbol{x}^T_i$. in fact, each row of the dataset is a data point $(\boldsymbol{x}^T_i,y_i)$ 
 - First we load the dataset, and then normalize each column of its input data (excluding the last column).
 - Next, the subgradient method is used for Elastic Net to estimate the parameters.
      - For deeper discussion on subgradient method, see our post in Repository **Optimization**.
 - Finally, we measure the accuracy of the model for *binary classification*.
 
**Hint:** There are better subgradient-based methods for *Elastic Net* and *Lasso* such as **Coordinate Descent** that we will discuss in the future. 
<hr>
The Python code at: https://github.com/ostad-ai/Machine-Learning
<br> Explanation: https://www.pinterest.com/HamedShahHosseini/Machine-Learning/

In [1]:
# importing required modules
import numpy as np
import pandas as pd

In [2]:
# loading the csv dataset
filepath='./diabetes.csv'
df=pd.read_csv(filepath)
rows,cols=df.shape
print(f'The dataset has {rows} rows and {cols} columns')
df.head() # showing top five rows

The dataset has 768 rows and 9 columns


,Pregnancies,Glucose,BloodPressure,SkinThickness,Insulin,BMI,DiabetesPedigreeFunction,Age,Outcome
0,6,148,72,35,0,33.6,0.627,50,1
1,1,85,66,29,0,26.6,0.351,31,0
2,8,183,64,0,0,23.3,0.672,32,1
3,1,89,66,23,94,28.1,0.167,21,0
4,0,137,40,35,168,43.1,2.288,33,1


In [3]:
# separating pairs X_i and y_i into matrix Xs and vector ys
# and then normalizing each data column separately, except the last column
ys=df['Outcome'].values
df_xs=df.drop(['Outcome'],axis=1)
df_xs=(df_xs-df_xs.mean())/df_xs.std()
Xs=df_xs.to_numpy()  #converting to a numpy array

In [4]:
# Subgradient method for Elastic Net (also for Lasso)
# Xs is a matrix with n rows and q-1 columns
# ys is a column vector of size n holding the dependent values yi
# etta0 is the learning rate constant
def subgradient_EN(Xs,ys,iter=1000,etta0=.01,landa1=5,landa2=1):
    X=np.ones((Xs.shape[0],Xs.shape[1]+1))
    X[:,1:]=Xs.copy()
    w=.1*np.random.rand(X.shape[1]).reshape(-1,1)
    for k in range(iter):
        gk=-X.T@(ys.reshape(-1,1)-X@w)
        gk+=landa1*np.sign(w)+landa2*w
        etta=etta0/np.sqrt(np.sum(gk**2))
        w-=etta*gk
    return w.flatten()

In the following cell, we use the subgradient method with its default $\lambda_1$ and $\lambda_2$.
However, you can change their values and observe the difference in parameters and/or accuracies.

In [5]:
# example
# estimated parameters for diabetes 
# because of L1 norm, 
# we see some (usually unneccesary) parameters are near to zero
w_hat=subgradient_EN(Xs,ys)
print(f'The estimated parameters:\n{w_hat}')

The estimated parameters:
[ 0.3420026   0.06483115  0.18184957 -0.03265791  0.00229541 -0.00719638
  0.09793499  0.04316883  0.02638846]


In [6]:
# measuring accuracy of the model for classification
# we use value of .5 to thereshold output to zero or one
X=np.ones((Xs.shape[0],Xs.shape[1]+1))
X[:,1:]=Xs.copy()
ys_hat=np.int16(X@w_hat.reshape(-1,1)>.5).flatten()  # estimated ys
accuracy=np.sum(ys_hat==ys)/len(ys)
print(f'The accuracy of the model for classification: {accuracy}')

The accuracy of the model for classification: 0.7721354166666666
